In [1]:
import pandas as pd

df = pd.read_csv('medical_cases_test.csv') #load dataset

# Calculate lengths
df['transcription_length'] = df['transcription'].apply(lambda x: len(str(x).split()))

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot histograms
plt.figure(figsize=(12,5))
sns.histplot(df['description_length'], bins=50, color='blue', alpha=0.6, label="Description")
sns.histplot(df['transcription_length'], bins=50, color='orange', alpha=0.6, label="Transcription")
plt.legend()
plt.title("Distribution of Document Lengths")
plt.xlabel("Number of Words")
plt.ylabel("Frequency")
plt.show()

KeyError: 'description_length'

<Figure size 1200x500 with 0 Axes>